<div style = "text-align: left"><font size = 8 color = "#0000FF"#0000FFface = "verdana"><b>Tesco customer churn project</b></font></div>
<div style = "text-align: leftt"><font><i>By Jerry Aremu</i></font></div>
<div style = "text-align: leftt"><font>28th February 2023</font></div>

### import requred libraries

In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_columns = 200

### notebook custom format

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.max_colwidth = None
pd.set_option("display.float_format", lambda x: '%.2f' % x)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



### Data Ingestion

In [4]:
data_folder = "data"
file_name = "telco_customer_churn.csv"

In [5]:
main_path = os.getcwd()

In [6]:
file_path = os.path.join(os.path.join(main_path, data_folder), file_name)
file_path

'C:\\Users\\jerea\\OneDrive\\Documents\\datahackerman_workspace\\telco_customer_churn\\data\\telco_customer_churn.csv'

In [7]:
telco_data = pd.read_csv(file_path)

In [8]:
telco_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### Data Inspection and Exploratory Analysis

#### Data inspection

In [9]:
# check the statistical summary of the data
telco_data.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.00,7043.00,7043.00
mean,0.16,32.37,64.76
std,0.37,24.56,30.09
min,0.00,0.00,18.25
25%,0.00,9.00,35.50
50%,0.00,29.00,70.35
75%,0.00,55.00,89.85
max,1.00,72.00,118.75


In [10]:
# check for data type mismatch and missing data
telco_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [11]:
# check the size of the data
telco_data.shape

(7043, 21)

#### Data exploration

##### Data Inspection findings:
1. Data appears to have been pre processed 
2. No missing data observed, and the 
3. object types for the data field appears correct


### Model developement

In [12]:
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

In [13]:
telco_data.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [14]:
# select all columns except the one that is to be predicted
X = telco_data[['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges']]

In [15]:
#drop the column id field
telco_data_drop = telco_data.drop(["customerID"], axis=1)

In [16]:
telco_data_drop.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [17]:
y = telco_data_drop["Churn"]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)

In [19]:
X_train.shape, X_test.shape

((4718, 20), (2325, 20))

### Train the model

In [33]:
train_data, test_data = train_test_split(telco_data_drop, test_size = 0.33, random_state = 42)

In [36]:
train_data.shape, test_data.shape

((4718, 20), (2325, 20))

In [37]:
label = "Churn"
print("Summary of class variable: \n", train_data[label].describe())


Summary of class variable: 
 count     4718
unique       2
top         No
freq      3477
Name: Churn, dtype: object


In [38]:
save_path = 'models'
predictor = TabularPredictor(label=label, path =save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "models\"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Train Data Rows:    4718
Train Data Columns: 19
Label Column: Churn
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  ['No', 'Yes']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = Yes, class 0 = No
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive (Yes) vs negative (No) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preproce

In [39]:
predictor.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.81,0.00,1.06,0.00,0.46,2,True,14
1,LightGBMXT,0.81,0.00,0.61,0.00,0.61,1,True,3
2,CatBoost,0.80,0.02,12.73,0.02,12.73,1,True,7
3,NeuralNetTorch,0.80,0.01,5.51,0.01,5.51,1,True,12
4,LightGBM,0.80,0.02,0.55,0.02,0.55,1,True,4
5,XGBoost,0.79,0.00,0.80,0.00,0.80,1,True,11
6,NeuralNetFastAI,0.79,0.02,6.48,0.02,6.48,1,True,10
7,LightGBMLarge,0.79,0.00,1.09,0.00,1.09,1,True,13
8,RandomForestGini,0.78,0.10,0.86,0.10,0.86,1,True,5
9,ExtraTreesGini,0.78,0.09,0.73,0.09,0.73,1,True,8


### Model evaluation

In [60]:
y_test= test_data[label] #values to predict

In [61]:
y_test[0:5]

185     Yes
2715     No
3825     No
1807    Yes
132      No
Name: Churn, dtype: object

In [63]:
save_path

'models'

In [64]:
predictor

In [65]:
# load the saved model
save_model_predictor = TabularPredictor.load(save_path) #not necessary just demonstrates how to load trained predictor from files

In [66]:
save_model_predictor

In [67]:
y_pred = save_model_predictor.predict(unlabelled_test_data)

In [68]:
y_pred[0:5]

185     Yes
2715     No
3825     No
1807    Yes
132      No
Name: Churn, dtype: object

In [72]:
perf = save_model_predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.7987096774193548
Evaluations on test data:
{
    "accuracy": 0.7987096774193548,
    "balanced_accuracy": 0.6785456913474134,
    "mcc": 0.4363255294295534,
    "f1": 0.528225806451613,
    "precision": 0.7197802197802198,
    "recall": 0.4171974522292994
}


In [73]:
save_model_predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetTorch,0.81,0.80,0.04,0.01,5.51,0.04,0.01,5.51,1,True,12
1,NeuralNetFastAI,0.80,0.79,0.07,0.02,6.48,0.07,0.02,6.48,1,True,10
2,LightGBM,0.80,0.80,0.02,0.02,0.55,0.02,0.02,0.55,1,True,4
3,LightGBMXT,0.80,0.81,0.01,0.00,0.61,0.01,0.00,0.61,1,True,3
4,WeightedEnsemble_L2,0.80,0.81,0.01,0.00,1.06,0.00,0.00,0.46,2,True,14
5,CatBoost,0.80,0.80,0.02,0.02,12.73,0.02,0.02,12.73,1,True,7
6,XGBoost,0.80,0.79,0.04,0.00,0.80,0.04,0.00,0.80,1,True,11
7,RandomForestEntr,0.79,0.77,0.21,0.09,0.67,0.21,0.09,0.67,1,True,6
8,RandomForestGini,0.79,0.78,0.21,0.10,0.86,0.21,0.10,0.86,1,True,5
9,ExtraTreesGini,0.79,0.78,0.23,0.09,0.73,0.23,0.09,0.73,1,True,8


### Features importance

In [74]:
%%time
save_model_predictor.feature_importance(train_data)

Computing feature importance via permutation shuffling for 19 features using 4718 rows with 5 shuffle sets...
	4.58s	= Expected runtime (0.92s per shuffle set)
	1.97s	= Actual runtime (Completed 5 of 5 shuffle sets)


CPU times: total: 5.83 s
Wall time: 1.99 s


,importance,stddev,p_value,n,p99_high,p99_low
tenure,0.05,0.00,0.00,5,0.05,0.04
Contract,0.03,0.00,0.00,5,0.04,0.02
InternetService,0.03,0.00,0.00,5,0.04,0.02
OnlineSecurity,0.02,0.00,0.00,5,0.03,0.02
PaperlessBilling,0.01,0.00,0.00,5,0.01,0.01
PaymentMethod,0.01,0.00,0.00,5,0.01,0.00
TechSupport,0.01,0.00,0.00,5,0.01,0.00
MultipleLines,0.00,0.00,0.00,5,0.01,0.00
PhoneService,0.00,0.00,0.00,5,0.00,0.00
StreamingMovies,0.00,0.00,0.01,5,0.00,-0.00


In [80]:
test_data["predicted_Churn"] = y_pred

In [81]:
test_data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,predicted_charges,predicted_Churn
185,Female,0,Yes,No,1,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,24.80,24.8,Yes,Yes,Yes
2715,Male,0,No,No,41,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Bank transfer (automatic),25.25,996.45,No,No,No
3825,Female,0,Yes,Yes,52,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.35,1031.7,No,No,No
1807,Female,0,No,No,1,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Electronic check,76.35,76.35,Yes,Yes,Yes
132,Male,0,No,No,67,Yes,No,DSL,No,No,No,Yes,No,No,Two year,No,Bank transfer (automatic),50.55,3260.1,No,No,No


In [ ]:
### Using input 

In [43]:
test_data["SeniorCitizen"].unique()

array([0, 1], dtype=int64)

In [44]:
test_data["MultipleLines"].unique()

array(['No phone service', 'Yes', 'No'], dtype=object)

In [45]:
test_data["tenure"].unique()

array([ 1, 41, 52, 67, 68, 23, 72, 70, 34, 28, 18, 15, 11, 37, 22, 46, 51,
        5, 45, 29, 50, 27, 60, 17, 33, 66, 69, 57,  8, 14, 25,  7, 13, 19,
       59,  9,  3, 10, 39, 54, 35, 12, 31, 53, 30, 48, 38, 65, 24, 40, 43,
       71, 61, 26, 42, 21, 56,  4,  6, 55,  2, 44, 62, 63, 49, 20, 16, 47,
       32, 64, 58, 36,  0], dtype=int64)

### Create a sample input

In [46]:
input_data_dict = {
    "gender": "female",
    "SeniorCitizen": 0,
    "Partner": "Yes", 
    "Dependents" : "No", 
    "tenure": 1,
    "PhoneService": "No",
    "MultipleLines": "No phone service",
    "InternetService": "DSL",
    "OnlineSecurity": "No",
    "OnlineBackup": "No",
    "DeviceProtection": "No",
    "TechSupport": "No",
    "StreamingTV": "No",
    "StreamingMovies": "No",
    "Contract": "Two year",
    "PaperlessBilling": "Yes",
    "PaymentMethod": "Electronic check",
    "MonthlyCharges": 24.8,
    "TotalCharges": 24.8
}

In [51]:
input_data_dict

{'gender': 'female',
 'SeniorCitizen': 0,
 'Partner': 'Yes',
 'Dependents': 'No',
 'tenure': 1,
 'PhoneService': 'No',
 'MultipleLines': 'No phone service',
 'InternetService': 'DSL',
 'OnlineSecurity': 'No',
 'OnlineBackup': 'No',
 'DeviceProtection': 'No',
 'TechSupport': 'No',
 'StreamingTV': 'No',
 'StreamingMovies': 'No',
 'Contract': 'Two year',
 'PaperlessBilling': 'Yes',
 'PaymentMethod': 'Electronic check',
 'MonthlyCharges': 24.8,
 'TotalCharges': 24.8}

In [52]:
input_data = pd.DataFrame([input_data_dict])

In [53]:
input_data

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,female,0,Yes,No,1,No,No phone service,DSL,No,No,No,No,No,No,Two year,Yes,Electronic check,24.80,24.80
